In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import glob
import numpy as np
from pathlib import Path
from collections import defaultdict

In [59]:
def class2rank(name: dict) -> list[int]:
    """
    """
    return [ord(c) - ord('a') + 1 for c in name]


def compute_rank_stats(path_annotations: Path | str, path_metadata: Path | str, key: str) -> dict:
    """
    """
    metadata = json.load(open(path_metadata))
    annotations = {}
    for filename in glob.glob(str(path_annotations / 'annotations/worker-response/iteration-1/*/*.json')):
        modelid = Path(filename).parent.stem
        annotations[int(modelid)] = json.load(open(filename))['answers']
    assert len(metadata) == len(annotations)

    method2ranks = {}
    for i in range(len(metadata)):
        ranks = []
        for ans in annotations[i]:
            ranks.append(class2rank(ans['answerContent']['crowd-classifier']['label']))
        ranks_average = []
        for j in range(len(ranks[0])):
            ranks_average.append(sum([r[j] for r in ranks]) / len(ranks))
        for method, index in metadata[i].items():
            method2ranks.setdefault(method, []).append(ranks_average[index])
    method2rank_stats = {
        method: {
            'avg': sum(ranks) / len(ranks),
            'std': np.std(ranks),
        } 
        for method, ranks in method2ranks.items()
    }
    return method2rank_stats

In [60]:
print(compute_rank_stats(
    Path('/home/ubuntu/meshseg/tests/mesh-segmentation-samesh-v-sdf-annotations'), 
    Path('/home/ubuntu/meshseg/tests/mesh_segmentation-samesh-v-sdf/metadata.json'),
    key='mesh-segmentation-high-quality-metadata'
))

{'combined': {'avg': 1.176, 'std': 0.2712145522152772}, 'shape_diameter_function': {'avg': 1.824, 'std': 0.2712145522152772}}


In [61]:
print(compute_rank_stats(
    Path('/home/ubuntu/meshseg/tests/mesh-segmentation-samesh-modalities-annotations'),
    Path('/home/ubuntu/meshseg/tests/mesh_segmentation-samesh-modalities/metadata.json'),
    key='mesh-segmentation-ablation-high-quality-metadata'
))

{'combined': {'avg': 2.3253333333333335, 'std': 0.5912908665698202}, 'matte': {'avg': 2.5866666666666664, 'std': 0.6946142014736589}, 'norm': {'avg': 2.5653333333333332, 'std': 0.577233247675688}, 'sdf': {'avg': 2.5226666666666664, 'std': 0.7503907130436931}}
